<a href="https://colab.research.google.com/github/RazerRaymond/MachineLearningPS/blob/main/Evaluating_Gesture_Recognition_using_NNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating Gesture Recognition using NNs

Data was based on WUSTL CSE 217A data in SP2019

## 1. Introduction

The data needed for this assignemnt can be found [here](https://wustl.box.com/s/q8mnl1o2zq2bh0ca5zajtk3msnu03ou8). All of it was gathered in `Homework 10 (Part I)`: 
- training
- validation
- augmented
- testing

Here are the neural network models trained on `training`:
- cse217_v1.h5 (still training; watch for announcement on Piazza)
- cse217_v2.h5 (still training; watch for announcement on Piazza)

Here are the neural network models trained on `augmented`:
- cse217_v1_augmented.h5 (still training; watch for announcement on Piazza)
- cse217_v2_augmented.h5 (still training; watch for announcement on Piazza)

Note that to train these models we used the `validation` dataset to determine when to stop the training process. 

## 2. Test Data Collection, Data Profiling, and Model Understanding

In this section, we will get a feel for our data.

In [ ]:
from os import makedirs, mkdir
from os.path import exists

base = 'utility/data'
raw = f'{base}/raw'
dirs = ['rock', 'paper', 'scissors']

if not exists(raw):
    makedirs(raw, exist_ok=True)

for sign in dirs:
    path = f'{raw}/{sign}'
    
    if not exists(path):
        mkdir(path)

Store the images you took of rocks (✊), papers (🤚), and scissors (✌️) in the correct folders in `utility/data/raw`. Then, run the following cell to produced rescaled images, which will be stored in `utility/data/testing`.

In [ ]:
import os
import warnings

from utility.util import load_image, resize_image, save_image


testing = f'{base}/testing'

for sign in dirs:
    path = f'{testing}/{sign}'
    
    if not exists(path):
        makedirs(path, exist_ok=True)

for path, _, files in os.walk(raw):
    sign = os.path.basename(path)

    for file in files:
        input_path = f'{path}/{file}'
        output_path = f'{testing}/{sign}/{file}'
        
        # note! warnings about lossy conversion are ok
        image = load_image(input_path)
        image = resize_image(image, (500, 500))

        save_image(output_path, image)

/home/ray/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ray/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ray/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ray/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarnin

utility/data/raw/scissors/4.jpg
utility/data/raw/scissors/2.jpg
utility/data/raw/scissors/5 (1).jpg
utility/data/raw/scissors/3.jpg
utility/data/raw/scissors/1.jpg


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


utility/data/raw/rock/4.jpg
utility/data/raw/rock/2.jpg
utility/data/raw/rock/5.jpg
utility/data/raw/rock/3.jpg


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


utility/data/raw/rock/1.jpg
utility/data/raw/paper/4.jpg
utility/data/raw/paper/5 (2).jpg


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


utility/data/raw/paper/2.jpg
utility/data/raw/paper/3.jpg
utility/data/raw/paper/1.jpg


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [ ]:
from tensorflow.keras.models import load_model

v1_raw = load_model('/home/ray/Downloads/hwhw10/utility/model.v1.raw.h5', compile=False)
v1_aug = load_model('/home/ray/Downloads/hwhw10/utility/model.v1.augmented.h5', compile=False)
v2_raw = load_model('/home/ray/Downloads/hwhw10/utility/model.v2.raw.h5', compile=False)
v2_aug = load_model('/home/ray/Downloads/hwhw10/utility/model.v2.augmented.h5', compile=False)

#v1_raw.summary()
#v1_aug.summary()
#v2_raw.summary()
#v2_aug.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## 3. Model Comparison: v1 vs v2

By now we should know all of the ins and outs about our datasets and models. Let's evaluate and compare the models. 

### None Augmented dataset

In the following cell, we provide an example of how to load the testing. Note the dimensions of the dataset (especially the size of the images).

In [ ]:
def accuracy(X, y, model):
    acc = 0
    label =  y
    image = X
    patchsize = model.input_shape[1]
    for i in range(0,len(X)):
        image_tran = skimage.transform.resize(image[i], (patchsize,patchsize))
        outs = model.predict(np.array([image_tran]))
        predicted = np.argmax(outs)
        if predicted == np.argmax(label[i]):
            acc+=1
    
    print("Number of pictures predicted correctly by model: %d" % acc)
    print("Number of picutres in the dataset: %d" % len(X))

    return acc/len(X)

In [ ]:
from utility.util import load_dataset
import skimage
import numpy as np

target_shape = (500, 500)
X_test_example, y_test_example = load_dataset('utility/data/testing', target_shape)
v1_t = accuracy(X_test_example, y_test_example, v1_raw) 
v2_t = accuracy(X_test_example, y_test_example, v2_raw) 
print("v1_raw on test: ")
print(v1_t)
print("v2_raw on test: ")
print(v2_t)

utility/data/testing/scissors/4.jpg
utility/data/testing/scissors/2.jpg
utility/data/testing/scissors/5 (1).jpg
utility/data/testing/scissors/3.jpg
utility/data/testing/scissors/1.jpg
utility/data/testing/rock/4.jpg
utility/data/testing/rock/2.jpg
utility/data/testing/rock/5.jpg
utility/data/testing/rock/3.jpg
utility/data/testing/rock/1.jpg
utility/data/testing/paper/4.jpg
utility/data/testing/paper/5 (2).jpg
utility/data/testing/paper/2.jpg
utility/data/testing/paper/3.jpg
utility/data/testing/paper/1.jpg
Number of pictures predicted correctly by model: 2
Number of picutres in the dataset: 15
Number of pictures predicted correctly by model: 5
Number of picutres in the dataset: 15
v1_raw on test: 
0.13333333333333333
v2_raw on test: 
0.3333333333333333


In [ ]:
target_shape = (500, 500)
X_train, y_train = load_dataset('/home/ray/Downloads/hwhw10/utility/data/training', target_shape)
v1_train = accuracy(X_train, y_train, v1_raw) 
v2_train = accuracy(X_train, y_train, v2_raw) 
print("v1_raw on train: ")
print(v1_train)
print("v2_raw on train: ")
print(v2_train)

In [ ]:
target_shape = (500, 500)
X_valid, y_valid = load_dataset('/home/ray/Downloads/hwhw10/utility/data/validation', target_shape)
v1_v = accuracy(X_valid, y_valid, v1_raw) 
v2_v = accuracy(X_valid, y_valid, v2_raw) 
print("v1_raw on validation: ")
print(v1_v)
print("v2_raw on validation: ")
print(v2_v)

In [ ]:
def create_user_testdata(path2folder, foldername):
    dataset_directory = pathlib.Path(path2folder)

    # Now check the data
    ddir=dataset_directory/foldername
    cdirs={}
    cdirs.update({ddir/"rock":0,
                  ddir/"paper":1,
                  ddir/"scissors":2})

    names = ["rock", "paper", "scissors"]

    for cdir,cdir_class in cdirs.items():
        assert cdir.exists()==1, str(cdir)+' does not exist'
        print("Found directory {} containing class {}".format(cdir,names[cdir_class]))

    imagesize = 500
    dataset1=[]
    for cdir,cn in reversed(list(cdirs.items())):

        for f in tqdm(list(cdir.glob("*"))):
            try:
                im=skimage.io.imread(f)
                h,w=im.shape[0:2] # height, width
                sz=min(h,w)
                im=im[(h//2-sz//2):(h//2+sz//2),(w//2-sz//2):(w//2+sz//2),:] # defines the central square
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    im=skimage.img_as_ubyte(skimage.transform.resize(im,(imagesize,imagesize))) # resize it to 500x500, whatever the original resolution
            except:
                warnings.warn("ignoring "+str(f))
                continue

            dataset1.append({
                "file": f,
                "label": cn,
                "image": im
            })

    print("Done")

    dataset1 = pd.DataFrame(dataset1)
    dataset1["dn"] = dataset1["file"].apply(lambda x: x.parent.parts[-2])
    return dataset1

# Show results by processing a single validataion or testing image
names = ["rock", "paper", "scissors"]

%matplotlib inline
def resultsShow(i, data, model):
    guide = { 0:"rock",1:"paper",2:"scissor"}
    d = data.iloc[i]
    im = d["image"]
    l = d["label"]
    fig,axs = plt.subplots(nrows=1,ncols=3,figsize=(15,5),gridspec_kw={'width_ratios':[1,1,0.5]})
    
    imt = imr = skimage.transform.resize(im, (model.input_shape[1],model.input_shape[1]))
    axs[0].imshow(im)
    axs[0].set_title("Image (true class: {})".format(names[l]))
    
    axs[1].imshow(imt,interpolation="nearest")
    axs[1].set_title("Network input")
    
    outs = model.predict(np.array([imt]))
    predicted = np.argmax(outs)
    print(outs)
    print("predicted label, %s" % guide.get(predicted))
    print("actual label, %s"% guide.get(l))

    axs[2].bar(np.array(range(len(names)))-0.5, outs[0,:], 1, color="gray")
    axs[2].set_ylim([0,1])
    axs[2].set_xticks(range(len(names)))
    axs[2].set_xticklabels(names)
    axs[2].set_ylabel("probability")
    axs[2].set_xlabel("class")
    axs[2].set_title("Network output")
    fig.tight_layout()
    plt.show()
    #fig.savefig("out_{:05d}_{}.png".format(i,("ok" if predicted==l else "ko")))    

In [ ]:
import pathlib
from tqdm import tqdm
import pandas as pd
data_eval = "raw"
base_d = pathlib.Path("/home/ray/Downloads/hwhw10/utility/data")
dataset_test = create_user_testdata(base_d,data_eval)


 60%|██████    | 3/5 [00:00<00:00, 29.24it/s]

Found directory /home/ray/Downloads/hwhw10/utility/data/raw/rock containing class rock
Found directory /home/ray/Downloads/hwhw10/utility/data/raw/paper containing class paper
Found directory /home/ray/Downloads/hwhw10/utility/data/raw/scissors containing class scissors


100%|██████████| 5/5 [00:00<00:00, 30.63it/s]

Done


In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
# Show results by processing a single validataion or testing image
import matplotlib.pyplot as plt
names = ["rock", "paper", "scissors"]

%matplotlib inline
def resultsShow(i, data, model):
    guide = { 0:"rock",1:"paper",2:"scissor"}
    d = data.iloc[i]
    im = d["image"]
    l = d["label"]
    fig,axs = plt.subplots(nrows=1,ncols=3,figsize=(15,5),gridspec_kw={'width_ratios':[1,1,0.5]})
    
    imt = imr = skimage.transform.resize(im, (model.input_shape[1],model.input_shape[1]))
    axs[0].imshow(im)
    axs[0].set_title("Image (true class: {})".format(names[l]))
    
    axs[1].imshow(imt,interpolation="nearest")
    axs[1].set_title("Network input")
    
    outs = model.predict(np.array([imt]))
    predicted = np.argmax(outs)
    print(outs)
    print("predicted label, %s" % guide.get(predicted))
    print("actual label, %s"% guide.get(l))

    axs[2].bar(np.array(range(len(names)))-0.5, outs[0,:], 1, color="gray")
    axs[2].set_ylim([0,1])
    axs[2].set_xticks(range(len(names)))
    axs[2].set_xticklabels(names)
    axs[2].set_ylabel("probability")
    axs[2].set_xlabel("class")
    axs[2].set_title("Network output")
    fig.tight_layout()
    plt.show()
    #fig.savefig("out_{:05d}_{}.png".format(i,("ok" if predicted==l else "ko")))    
print("Results on individual {} inputs: ".format(dataset_test.loc[0].dn)) 
interact(resultsShow, i=widgets.IntSlider(min=0,max=len(dataset_test)-1, step=1, value=0, continuous_update=False), data=fixed(dataset_test.sample(len(dataset_test))), model=fixed(v2_raw))



Results on individual raw inputs: 


interactive(children=(IntSlider(value=0, continuous_update=False, description='i', max=14), Output()), _dom_cl…

<function __main__.resultsShow(i, data, model)>

## 4. Model Comparison: original vs augmented
- Did data augemntation help? 
- Which of the two NN versions benefited or suffered more from data augmentation? 
- Give an explanation/guestimate why this is the case.

In [ ]:
v1_t_aug = accuracy(X_test_example, y_test_example, v1_aug) 
v2_t_aug = accuracy(X_test_example, y_test_example, v2_aug) 
print("v1_aug on test: ")
print(v1_t_aug)
print("v2_aug on test: ")
print(v2_t_aug)
#v1_train_aug = accuracy(X_train, y_train, v1_aug) 
#v2_train_aug = accuracy(X_train, y_train, v2_aug) 
#print("v1_aug on train: ")
#print(v1_train_aug)
#print("v2_aug on train: ")
#print(v2_train_aug)
#v1_v_aug = accuracy(X_valid, y_valid, v1_aug) 
#v2_v_aug = accuracy(X_valid, y_valid, v2_aug) 
#print("v1_aug on validation: ")
#print(v1_v_aug)
#print("v2_aug on validation: ")
#print(v2_v_aug)

Number of pictures predicted correctly by model: 6
Number of picutres in the dataset: 15
Number of pictures predicted correctly by model: 8
Number of picutres in the dataset: 15
v1_aug on test: 
0.4
v2_aug on test: 
0.5333333333333333
